In [1]:

%load_ext autoreload
%autoreload 2
from loaders.loaders import *
#from dataset_dict import dataset_dict, output_path
#from loaders.RecordDistributor import RecordDistributor
#from loaders.CleanedEnglishDataset import WikiBookEn
from loaders.OtherDatasets import *
#from loaders.PilePythonDataset import PilePythonDataset

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from loaders.RandomEnglishDataset import RandomEnglishDataset
#ds= load_dataset("suolyer/pile_stackexchange",streaming=True,split="validation")
#ds = load_dataset("Muennighoff/flan",streaming=True,split="train")
ds=RandomEnglishDataset()

loading many datasets. This may take a while...


/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for allenai/peS2o contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/peS2o
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `

python0000.jsonl.zst already exists.
python0001.jsonl.zst already exists.
python0002.jsonl.zst already exists.
python0003.jsonl.zst already exists.
python0004.jsonl.zst already exists.
python0005.jsonl.zst already exists.
python0006.jsonl.zst already exists.
python0007.jsonl.zst already exists.
python0008.jsonl.zst already exists.
python0009.jsonl.zst already exists.
python0010.jsonl.zst already exists.
python0011.jsonl.zst already exists.
python0012.jsonl.zst already exists.
python0013.jsonl.zst already exists.
python0014.jsonl.zst already exists.
python0015.jsonl.zst already exists.
python0016.jsonl.zst already exists.
python0017.jsonl.zst already exists.
python0018.jsonl.zst already exists.
python0019.jsonl.zst already exists.
python0020.jsonl.zst already exists.
python0021.jsonl.zst already exists.
python0022.jsonl.zst already exists.
python0023.jsonl.zst already exists.
python0024.jsonl.zst already exists.
python0025.jsonl.zst already exists.
python0026.jsonl.zst already exists.
p

In [4]:

loader=iter(ds)

In [7]:
import json
out_path="../data/eng/eng.jsonl"
out_path="/data/hatakeyama/python/eng_corpus/eng.jsonl"
from tqdm import tqdm
for i in tqdm(range(10**5)):
    t=next(loader)["text"]
    if t=="":
        continue

    with open(out_path,"a") as f:
        f.write(json.dumps({"text":t},ensure_ascii=False)+"\n")

100%|██████████| 100000/100000 [01:04<00:00, 1547.57it/s]
